## MF-DGP Improved

In [ ]:
import tensorflow as tf
import gpflow
import numpy as np
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
from dgp_dace.models.MF_DGP import MultiFidelityDeepGP
from gpflow import set_trainable
from gpflow.optimizers import NaturalGradient
from sklearn.metrics import mean_squared_error, r2_score
import scipy
from prettytable import PrettyTable


In [ ]:
from emukit.test_functions.multi_fidelity import (multi_fidelity_borehole_function, multi_fidelity_branin_function,
                                                  multi_fidelity_park_function, multi_fidelity_hartmann_3d,
                                                  multi_fidelity_currin_function)
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.experimental_design.model_free.latin_design import LatinDesign

In [ ]:
from collections import namedtuple
Function = namedtuple('Function', ['name', 'y_scale', 'noise_level', 'do_x_scaling', 'num_data', 'fcn'])
park = Function(name='park', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[30, 5], 
                    fcn=multi_fidelity_park_function)

In [4]:
def generate_data(fcn_tuple, n_test_points):
    """
    Generates train and test data for
    """
    # A different definition of the parameter space for the branin function was used in the paper
    if fcn_tuple.name == 'branin':
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace([ContinuousParameter('x1', -5., 0.), ContinuousParameter('x2', 10., 15.)])
    else:
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace(space._parameters[:-1])
    do_x_scaling = fcn_tuple.do_x_scaling
    # Generate training data
    latin = LatinDesign(new_space)
    X = [latin.get_samples(n) for n in fcn_tuple.num_data]
    # Scale X if required
    if do_x_scaling:
        scalings = X[0].std(axis=0)
    else:
        scalings = np.ones(X[0].shape[1])
    for x in X:
        x /= scalings
    Y = []
    for i, x in enumerate(X):
        Y.append(fcn.f[i](x * scalings))
    y_scale = fcn_tuple.y_scale
    # scale y and add noise if required
    noise_levels = fcn_tuple.noise_level
    for y, std_noise in zip(Y, noise_levels):
        y /= y_scale + std_noise * np.random.randn(y.shape[0], 1)
    # Generate test data
    x_test = latin.get_samples(n_test_points)
    x_test /= scalings
    y_test = fcn.f[-1](x_test * scalings)
    y_test /= y_scale
    print(X[1].shape)
    return x_test, y_test, X, Y

def print_metrics(metrics):
    model_names = list(list(metrics.values())[0].keys())
    metric_names = ['r2', 'mnll', 'rmse']
    table = PrettyTable(['model'] + metric_names)
    for name in model_names:
        mean = []
        for metric_name in metric_names:
            mean.append(np.mean([metric[name][metric_name] for metric in metrics.values()]))
        table.add_row([name] + mean)
    print(table)

def do_benchmark(fcn_tuple):
    metrics = dict()
    # Some random seeds to use
    seeds = [123, 184, 202, 289, 732,127, 1847, 2022, 2879, 7432,4123, 1084, 2052, 22289, 7132,1203, 15784, 2, 89, 32]
    for i, seed in enumerate(seeds):
        run_name = str(seed) + str(fcn_tuple.num_data)
        metrics[run_name] = test_function(fcn_tuple, seed)
        print('After ' + str(i+1) + ' runs of ' + fcn_tuple.name)
        print_metrics(metrics)
    return metrics



def benchmark_models(models, x_test, y_test):
    metrics = dict()
    for model in models:
        model.optimize_nat_adam(lr_adam = 0.01,lr_gamma = 0.01,iterations1=1000,iterations2=2000 ,iterations3=6000)
        y_mean, y_var = model.predict(x_test)
        metrics[model.name] = calculate_metrics(model,y_test, y_mean, y_var)
        print('+ ######################## +')
        print(model.name, 'r2', metrics[model.name]['r2'])
        print('+ ######################## + ')
    return metrics

def test_function(fcn, seed):
    np.random.seed(seed)
    x_test, y_test, X, Y = generate_data(fcn, 1000)
    mf_dgp_fix_lf_mean = MultiFidelityDeepGP(X, Y, n_iter=5000)
    models = [mf_dgp_fix_lf_mean]
    return benchmark_models(models, x_test, y_test)

def calculate_metrics(model,y_test, y_mean_prediction, y_var_prediction):
    # R2
    r2 = r2_score(y_test, y_mean_prediction)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_mean_prediction))
    # Test log likelihood
    mnll = -np.sum(scipy.stats.norm.logpdf(y_test, loc=y_mean_prediction, scale=np.sqrt(y_var_prediction)))/len(y_test)
    return {'r2': r2, 'rmse': rmse, 'mnll': mnll}

### Example one repetition

In [ ]:
sd = 123
np.random.seed(sd)
x_test, y_test, X, Y = generate_data(park, 1000)
model = MultiFidelityDeepGP(X, Y, n_iter=5000)
model.optimize_nat_adam(lr_adam= 0.001,iterations1=1000,iterations2=2000 ,iterations3=6000)

In [ ]:
from gpflow.utilities import print_summary

In [ ]:
print_summary(model,'notebook')

In [ ]:
y_mean, y_var = model.predict(x_test)
metrics = calculate_metrics(model,y_test, y_mean, y_var)
print(metrics)

### Example multiple repetition

In [5]:
metrics_park = []
metrics_park.append(do_benchmark(park))
pickle.dump(metrics_park[0],open('metrics_park_code_.p',"wb"))
print('Park')
print_metrics(metrics_park[0])

(5, 4)
4
ELBO: -1057886.0957877925
ELBO: -88507.17706362682
ELBO: -32850.25156663193
ELBO: -15114.084859131885
ELBO: -9168.142401072295
ELBO: -5752.985106658471
ELBO: -3873.8404749975152
ELBO: -274.31075723745295
ELBO: -231.85898471607427
ELBO: -203.5521753735586
ELBO: -180.51632210645198
ELBO: -161.5576418846613
ELBO: -145.80352303099588
ELBO: -132.40997677648318
ELBO: -121.59405417518937
ELBO: -112.09195244636965
ELBO: -103.58665869677499
ELBO: -97.31284656043353
+ ######################## +
mf_dgp r2 0.9869947446470132
+ ######################## + 
After 1 runs of park
+--------+--------------------+--------------------+--------------------+
| model  |         r2         |        mnll        |        rmse        |
+--------+--------------------+--------------------+--------------------+
| mf_dgp | 0.9869947446470132 | 0.9667126413059297 | 0.5490408715209518 |
+--------+--------------------+--------------------+--------------------+
(5, 4)
4
ELBO: -1024878.2842969666
ELBO: -85746.747

ELBO: -1145549.4882049172
ELBO: -95847.49238078827
ELBO: -35576.3583774688
ELBO: -16529.73476333261
ELBO: -9363.172696764253
ELBO: -6251.151054302895
ELBO: -4395.909539743785
ELBO: -288.4197803787507
ELBO: -234.91015264584885
ELBO: -207.06124059568967
ELBO: -188.07045396182787
ELBO: -173.34002789939893
ELBO: -160.87162184370564
ELBO: -149.69377322398753
ELBO: -139.31888577249694
ELBO: -129.53912044665856
ELBO: -120.61067887635262
ELBO: -112.33920014856213
+ ######################## +
mf_dgp r2 0.4484734678758362
+ ######################## + 
After 10 runs of park
+--------+--------------------+-------------------+--------------------+
| model  |         r2         |        mnll       |        rmse        |
+--------+--------------------+-------------------+--------------------+
| mf_dgp | 0.9339713863581329 | 1.324131686197588 | 0.8275950884702681 |
+--------+--------------------+-------------------+--------------------+
(5, 4)
4
ELBO: -495311.2753267885
ELBO: -41477.603886404635
ELBO:

KeyboardInterrupt: 

In [ ]:
sd = 123
np.random.seed(sd)
x_test, y_test, X, Y = generate_data(park, 1000)
model = MultiFidelityDeepGP(X, Y, n_iter=5000)
model.optimize_nat_adam(lr_adam= 0.001,iterations1=1000,iterations2=2000 ,iterations3=6000)

In [14]:
from gpflow.utilities import print_summary

In [15]:
print_summary(model,'notebook')

name,class,transform,prior,trainable,shape,dtype,value
MultiFidelityDeepGP.model.layers[0].q_mu,Parameter,Identity,,True,"(30, 1)",float64,[[17.86954844...
MultiFidelityDeepGP.model.layers[0].q_sqrt,Parameter,FillTriangular,,True,"(1, 30, 30)",float64,"[[[5.44498600e-01, 0.00000000e+00, 0.00000000e+00..."
MultiFidelityDeepGP.model.layers[0].feature.Z,Parameter,Identity,,True,"(30, 4)",float64,"[[-0.11235509, 0.51653229, 0.88960496..."
MultiFidelityDeepGP.model.layers[0].kern.kernels[0].variance,Parameter,Softplus,,True,(),float64,2.7267379159209115
MultiFidelityDeepGP.model.layers[0].kern.kernels[0].lengthscales,Parameter,Softplus,,True,"(4,)",float64,"[0.92242517, 1.10050614, 1.42698475..."
MultiFidelityDeepGP.model.layers[0].kern.kernels[1].variance,Parameter,Softplus,,True,(),float64,2.140243393012561e-06
MultiFidelityDeepGP.model.layers[1].q_mu,Parameter,Identity,,True,"(5, 1)",float64,[[3.21672...
MultiFidelityDeepGP.model.layers[1].q_sqrt,Parameter,FillTriangular,,True,"(1, 5, 5)",float64,"[[[1.01166395, 0., 0...."
MultiFidelityDeepGP.model.layers[1].feature.Z_left,Parameter,Identity,,True,"(5, 4)",float64,"[[-1.48292839, 0.62005329, 2.84429447..."
MultiFidelityDeepGP.model.layers[1].kern.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,0.15306218138395858


In [46]:
y_mean, y_var = model.predict(x_test)
metrics = calculate_metrics(model,y_test, y_mean, y_var)
print(metrics)

{'r2': 0.984671418573173, 'rmse': 0.5960685609072209, 'mnll': 1.0616846163811964}
